# Data Extraction

### Import the required libraries


In [1]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

###  File Handling

In [2]:
import os

def handle_file(path: str, name: str) -> str:
    """
    Rename any conflicting file to prevent unwanted overriding.
    """

    file_path = os.path.join(path, name)

    if os.path.isfile(file_path):
        print("Warning!")
        print("A file exists with the same name as the one being saved.")
        keep = input("Should I override the existing file? (y/n): ").strip().lower()

        if keep != 'y' and keep != 'yes':
            new_name = f"{name}.backup"
            new_file_path = os.path.join(path, new_name)
            print(f"Keeping existing file as {new_file_path}")
            os.rename(file_path, new_file_path)

    return file_path

Insights
* File Conflict Handling: The code effectively prevents accidental file overwrites by checking for existing files with the same name. If a conflict is detected, it renames the existing file with a ".backup" extension, ensuring data preservation.

* User Interaction: It engages the user by asking for confirmation before overwriting a file, enhancing user experience and preventing unintentional data loss.

### [Housing Construction Spending](https://www.census.gov/construction/c30/c30index.html)

In [3]:
# Read the Excel file from the given URL and skip the first two rows.
constr_spend = pd.read_excel('https://www.census.gov/construction/c30/xls/residentialsa.xls', skiprows=[0, 1])

# Restrict the data to records up to 2022 only as 2023 has lot of missing values
constr_spend = constr_spend.iloc[0:30, :]

# Automatically convert the data types of columns to their suitable types.
constr_spend = constr_spend.convert_dtypes()


Priv-Res SA


Insights
* The code reads an Excel file from a URL, skipping the first two rows, and stores it as a Pandas DataFrame. It then selects data up to the year 2022, excluding 2023, known for missing values. Finally, it automatically converts the DataFrame's column data types to their most suitable types.

Top 5 values in the constrution spending

In [4]:
constr_spend.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1993,194150,194689,190185,198296,198835,203175,208760,212238,214446,214778,227235,235648
1,1994,238612,236577,235150,244746,244056,244032,245463,243134,242619,234905,239857,241395
2,1995,241184,234257,228013,225488,222062,218972,224113,226045,228688,227589,231913,235572
3,1996,240958,243695,249952,252836,259296,263085,263549,263618,265677,261444,259668,253234
4,1997,255365,258595,264566,257490,264581,262299,261593,263951,270511,272980,272786,269192


#### Parse the data into an organized structure

In [5]:
constr_spend = constr_spend.set_index('Year').stack().reset_index().rename(columns={'Year':'year', 'level_1':'month', 0:'spend'})
constr_spend.head()

,year,month,spend
0,1993,Jan,194150
1,1993,Feb,194689
2,1993,Mar,190185
3,1993,Apr,198296
4,1993,May,198835


Insights
* The code reshapes a DataFrame by setting 'Year' as the index, stacking columns into rows, and then renaming the columns to 'year', 'month', and 'spend'. This transformation organizes data into a long format suitable for analysis.

In [6]:
# Combine 'year' and 'month' columns into a 'date' column as a string.
constr_spend['date'] = constr_spend['year'].astype(str) + '-' + constr_spend['month']

# Convert the 'date' column to a datetime format.
constr_spend['date'] = pd.to_datetime(constr_spend['date'])

# Drop the 'year' and 'month' columns.
constr_spend.drop(['year', 'month'], axis=1, inplace=True)

# Set the 'date' column as the DataFrame's index.
constr_spend.set_index(['date'], inplace=True)

# Display the updated DataFrame.
constr_spend.head()


,spend
date,
1993-01-01,194150
1993-02-01,194689
1993-03-01,190185
1993-04-01,198296
1993-05-01,198835


In [7]:
hud_supply_df = pd.read_excel('https://www.huduser.gov/portal/periodicals/ushmc/charts/HousingSupply.xlsx',
                        sheet_name='Construction_Monthly')

NewForSale_Annual
NewForSale_Monthly


In [8]:
hud_supply_df

,Month,Auth_US,Auth_S1,Auth_S2to4,Auth_S5more,Auth_NE,Auth_MW,Auth_S,Auth_W,Starts_US,...,UndCon_MW,UndCon_S,UndCon_W,Complt_US,Complt_S1,Complt_S5more,Complt_NE,Complt_MW,Complt_S,Complt_W
0,1968-01-01,1179,679,81,419,181,333,414,251,1380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1968-02-01,1342,711,89,542,224,381,460,277,1520,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1968-03-01,1370,691,80,599,297,332,454,287,1466,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1968-04-01,1286,675,81,530,243,330,446,267,1554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1968-05-01,1297,665,83,549,218,369,435,275,1408,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661,2023-02-01,1482,796,48,638,113,188,825,356,1436,...,211.0,787.0,469.0,1577.0,1029.0,542.0,131.0,205.0,881.0,360.0
662,2023-03-01,1437,829,52,556,148,204,768,317,1380,...,214.0,787.0,462.0,1528.0,1035.0,480.0,135.0,215.0,761.0,417.0
663,2023-04-01,1417,856,58,503,107,174,803,333,1348,...,210.0,796.0,460.0,1416.0,979.0,420.0,136.0,211.0,734.0,335.0
664,2023-05-01,1496,902,54,540,137,187,822,350,1559,...,209.0,799.0,457.0,1518.0,1014.0,488.0,115.0,205.0,863.0,335.0


In [9]:
## remove useless rows and columns

permits = hud_supply_df[['Month', 'Auth_S1']].copy()

permits.columns = ['date', 'permits']

In [10]:
## date column is already in datetime format

permits['date'] = pd.to_datetime(permits['date'])

permits['date'] = permits['date'].apply(lambda dt: dt.replace(day=1))

permits.set_index(['date'], inplace=True)

Insights
* The code takes a DataFrame column named 'date' containing datetime values, replaces the day component with 1 for each datetime, and sets the modified 'date' column as the index of the DataFrame 'permits'.

In [11]:
permits

,permits
date,
1968-01-01,679
1968-02-01,711
1968-03-01,691
1968-04-01,675
1968-05-01,665
...,...
2023-02-01,796
2023-03-01,829
2023-04-01,856


### [New construction of residential properties commences](https://www.huduser.gov/Portal/ushmc/hs_sfm.html)

In [12]:
new_starts = hud_supply_df[['Month', 'Starts_S1']].copy()

new_starts.columns = ['date', 'starts']

new_starts['date'] = new_starts['date'].apply(lambda dt: dt.replace(day=1))

new_starts.set_index(['date'], inplace=True)

Insights
* The code extracts the 'Month' and 'Starts_S1' columns from a DataFrame called 'hud_supply_df', renames the columns to 'date' and 'starts', and converts the 'date' values to the first day of each month. Finally, it sets the 'date' column as the index of the DataFrame 'new_starts'.

In [13]:
new_starts

,starts
date,
1968-01-01,839
1968-02-01,993
1968-03-01,912
1968-04-01,925
1968-05-01,850
...,...
2023-02-01,835
2023-03-01,843
2023-04-01,847


### [Compeletion of House](https://www.huduser.gov/Portal/ushmc/hs_sfm.html)

In [14]:
# Extract the 'Month' and 'Complt_S1' columns from the DataFrame 'hud_supply_df' and create a copy.
completions = hud_supply_df[['Month', 'Complt_S1']].copy()

# Rename the columns of the new DataFrame to 'date' and 'completions'.
completions.columns = ['date', 'completions']

# Convert the 'date' values to the first day of each month using a lambda function.
completions['date'] = completions['date'].apply(lambda dt: dt.replace(day=1))

# Set the 'date' column as the index of the DataFrame 'completions'.
completions.set_index(['date'], inplace=True)

Insights
* The above creates a new DataFrame named 'completions' by copying the 'Month' and 'Complt_S1' columns from 'hud_supply_df,' renaming them, and converting the 'date' values to the first day of each month. It then sets the 'date' column as the index of the 'completions' DataFrame, essentially organizing the data by month for completions.

In [15]:
completions

,completions
date,
1968-01-01,NaN
1968-02-01,NaN
1968-03-01,NaN
1968-04-01,NaN
1968-05-01,NaN
...,...
2023-02-01,1029.0
2023-03-01,1035.0
2023-04-01,979.0


### [Manufacturing of House](https://www.huduser.gov/portal/ushmc/hs_man_hsg.html)

In [16]:
manufacturing = pd.read_excel('https://www.huduser.gov/portal/periodicals/ushmc/charts/HousingSupply.xlsx',
                        sheet_name='Manufactured_Monthly')

NewForSale_Annual
NewForSale_Monthly


In [17]:
manufacturing = manufacturing.loc[:, ['Month', 'Shipments_US']]

manufacturing.columns = ['date', 'manufactured']

manufacturing['date'] = manufacturing['date'].apply(lambda dt: dt.replace(day=1))
manufacturing.set_index(['date'], inplace=True)

In [18]:
manufacturing

,manufactured
date,
1978-01-01,293
1978-02-01,275
1978-03-01,277
1978-04-01,267
1978-05-01,276
...,...
2023-02-01,85
2023-03-01,83
2023-04-01,84


Insights
* This is an Excel file from a URL, extracts specific columns ('Month' and 'Shipments_US'), renames the columns, converts the 'date' column to the first day of each month, and sets 'date' as the index for further analysis, likely related to manufacturing shipment data.

### [Building Permit Value](https://www.census.gov/construction/bps/index.html)

In [19]:
building_permit_value = pd.read_excel('https://www.census.gov/construction/bps/permitsbyusreg_cust.xls',
                             sheet_name='Val Monthly',
                            skiprows=list(range(4)), header=[0,1])

In [20]:
## drop unnecessary columns

building_permit_value = building_permit_value.iloc[:-6, [1, 3]]

building_permit_value.columns = ['date', 'permit_val']

In [21]:
## parse datetime

building_permit_value['date'] = pd.to_datetime(building_permit_value['date'], format='%Y-%m-%d')

building_permit_value['date'] = building_permit_value['date'].apply(lambda dt: dt.replace(day=1))

building_permit_value.set_index(['date'], inplace=True)

In [22]:
building_permit_value

,permit_val
date,
1988-01-01,4081.302
1988-02-01,5050.834
1988-03-01,7787.515
1988-04-01,7595.054
1988-05-01,8119.532
...,...
2023-01-01,15487.041
2023-02-01,16513.078
2023-03-01,22502.037


Insights
* This code first parses a 'date' column in a DataFrame to datetime format with the specified format ('%Y-%m-%d'), then replaces the day component with 1 for each date and sets the 'date' column as the DataFrame's index, likely for time series analysis with monthly granularity.

### [New House for Sale](https://www.huduser.gov/portal/ushmc/hs_newsf.html)

In [23]:
new_for_sale = pd.read_excel('https://www.huduser.gov/portal/periodicals/ushmc/charts/HousingSupply.xlsx',
                        sheet_name='NewForSale_Monthly')

NewForSale_Annual
NewForSale_Monthly


In [24]:
# Selecting specific columns 'Month', 'NewForSale_US', and 'NewMnSply_US'
new_for_sale = new_for_sale.loc[:, ['Month', 'NewForSale_US', 'NewMnSply_US']]

# Renaming columns for clarity
new_for_sale.columns = ['date', 'new_for_sale', 'months_supply']

# Replacing the day component of the 'date' column with 1 for each date
new_for_sale['date'] = new_for_sale['date'].apply(lambda dt: dt.replace(day=1))

# Setting the 'date' column as the index for time series analysis
new_for_sale.set_index(['date'], inplace=True)

In [25]:
new_for_sale

,new_for_sale,months_supply
date,,
1963-01-01,234,4.7
1963-02-01,236,6.6
1963-03-01,246,6.4
1963-04-01,254,5.3
1963-05-01,254,5.1
...,...,...
2023-02-01,437,8.4
2023-03-01,433,8.1
2023-04-01,432,7.7


Insights
* This code processes a DataFrame by selecting specific columns, renaming them, converting the 'date' column to the first day of each month, and setting 'date' as the index. It's likely preparing the data for time series analysis of new property listings and monthly supply.

### [Housing Vacancies Rate](https://www.census.gov/housing/hvs/data/histtabs.html) Quarterly

In [26]:
vacancy_df = pd.read_excel('https://www.census.gov/housing/hvs/files/qtr222/tab1.xlsx',
                       skiprows=range(4), header=[0,1,2])

In [27]:
# Dropping the second level of column labels, if present
vacancy_df.columns = vacancy_df.columns.droplevel(1)

# Slicing the DataFrame to exclude the first 3 rows and last 7 rows
vacancy_df = vacancy_df.iloc[3:-7]

# Creating a multi-level column index with arrays 'rent_vacancy' and 'owner_vacancy' for the first level,
# and 'Q1', 'Q2', 'Q3', 'Q4' for the second level
arrays = [['rent_vacancy', 'owner_vacancy'], ['Q1', 'Q2', 'Q3', 'Q4']]

# Creating a MultiIndex from the arrays and inserting a 'year' column at the beginning
columns = pd.MultiIndex.from_product(arrays)
columns = columns.insert(0, ('year', ''))

# Assigning the newly created MultiIndex to the DataFrame columns
vacancy_df.columns = columns

vacancy_df

year rent_vacancy                owner_vacancy            \
                                    Q1   Q2   Q3   Q4            Q1   Q2   Q3   
3        2021...…………..…….          6.8  6.2  5.8  5.6           0.9  0.9  0.9   
4        2020...…………..…….          6.6  5.7  6.4  6.5           1.1  0.9  0.9   
5                     NaN          NaN  NaN  NaN  NaN           NaN  NaN  NaN   
6        2019...…………..…….          7.0  6.8  6.8  6.4           1.4  1.3  1.4   
7        2018...…………..…….          7.0  6.8  7.1  6.6           1.5  1.5  1.6   
..                    ...          ...  ...  ...  ...           ...  ...  ...   
65  1969.....…………........          5.6  5.7  5.5  5.1           1.0  1.0  1.1   
66  1968..…………...........          6.1  6.2  5.9  5.4           1.1  1.1  1.2   
67  1967..…………...........          7.3  6.9  7.0  6.2           1.4  1.3  1.4   
68  1966..…………...........          8.3  7.4  7.4  7.7           1.5  1.5  1.4   
69  1965..…………...........          8.5  8.2  7.8  8.5           1.7  1.5  1.6   

         
     Q4  
3   0.9  
4   1.0  
5   NaN  
6   1.4  
7   1.5  
..  ...  
65  1.0  
66  1.2  
67  1.3  
68  1.3  
69  1.5  

[67 rows x 9 columns]

In [28]:
vacancy_df.dropna(axis=0, how='all', inplace=True)

mask = vacancy_df['year'].str.find('r') !=-1

repeat_cols = vacancy_df['year'][mask].str[:4].values

drop_index = vacancy_df[vacancy_df['year'].apply(lambda x: str(x)[:4] in repeat_cols and str(x).find('r') == -1)].index


vacancy_df = vacancy_df.drop(axis=0, index=drop_index).reset_index(drop=True)

vacancy_df['year'] = vacancy_df['year'].str[:4]

vacancy_df = vacancy_df.convert_dtypes()

vacancy_df.sort_values(['year'], ascending=True, inplace=True, ignore_index=True)

In [29]:
vacancy_df

year rent_vacancy                   owner_vacancy               
                   Q1    Q2    Q3    Q4            Q1   Q2   Q3   Q4
0   1965          8.5   8.2   7.8   8.5           1.7  1.5  1.6  1.5
1   1966          8.3   7.4   7.4   7.7           1.5  1.5  1.4  1.3
2   1967          7.3   6.9   7.0   6.2           1.4  1.3  1.4  1.3
3   1968          6.1   6.2   5.9   5.4           1.1  1.1  1.2  1.2
4   1969          5.6   5.7   5.5   5.1           1.0  1.0  1.1  1.0
5   1970          5.4   5.4   5.3   5.2           1.0  1.0  1.1  1.1
6   1971          5.3   5.3   5.6   5.6           1.0  0.9  1.0  1.0
7   1972          5.3   5.5   5.8   5.6           1.0  1.0  0.9  1.0
8   1973          5.7   5.8   5.8   5.8           1.0  0.9  1.1  1.2
9   1974          6.2   6.3   6.2   6.0           1.2  1.1  1.2  1.3
10  1975          6.1   6.3   6.2   5.4           1.2  1.2  1.4  1.2
11  1976          5.5   5.8   5.7   5.3           1.2  1.2  1.3  1.2
12  1977          5.1   5.3   5.4   5.1           1.3  1.3  1.1  1.0
13  1978          5.0   5.1   5.0   5.0           1.0  0.9  1.0  1.1
14  1979          5.1   5.5   5.7   5.4           1.1  1.1  1.2  1.3
15  1980          5.2   5.6   5.7   5.0           1.3  1.4  1.4  1.4
16  1981          5.2   5.0   5.0   5.0           1.3  1.3  1.5  1.4
17  1982          5.3   5.1   5.3   5.5           1.4  1.6  1.5  1.6
18  1983          5.7   5.5   5.8   5.5           1.4  1.5  1.6  1.6
19  1984          5.6   5.5   6.0   6.3           1.6  1.7  1.7  1.7
20  1985          6.3   6.2   6.8   6.7           1.8  1.9  1.8  1.6
21  1986          6.9   7.3   7.5   7.7           1.5  1.7  1.6  1.6
22  1987          7.4   7.5   8.1   7.8           1.7  1.7  1.7  1.6
23  1988          8.0   7.7   7.8   7.3           1.6  1.6  1.6  1.6
24  1989          7.5   7.4   7.6   7.1           1.7  1.7  1.9  1.8
25  1990          7.5   7.0   7.2   7.2           1.7  1.7  1.7  1.7
26  1991          7.5   7.3   7.6   7.3           1.7  1.8  1.8  1.6
27  1992          7.4   7.7   7.3   7.1           1.5  1.6  1.6  1.5
28  1993          7.8   7.6   7.0   6.9           1.4  1.4  1.4  1.4
29  1994          7.5   7.4   7.2   7.4           1.4  1.4  1.4  1.6
30  1995          7.4   7.7   7.7   7.7           1.5  1.6  1.5  1.6
31  1996          7.9   7.8   8.0   7.7           1.6  1.5  1.7  1.7
32  1997          7.5   7.9   7.9   7.7           1.7  1.6  1.5  1.7
33  1998          7.7   8.0   8.2   7.8           1.7  1.7  1.7  1.8
34  1999          8.2   8.1   8.2   7.9           1.8  1.6  1.6  1.6
35  2000          7.9   8.0   8.2   7.8           1.6  1.5  1.6  1.6
36  2001          8.2   8.3   8.4   8.8           1.5  1.8  1.9  1.8
37  2002          9.1   8.4   9.0   9.3           1.7  1.7  1.7  1.7
38  2003          9.4   9.6   9.9  10.2           1.7  1.7  1.9  1.8
39  2004         10.4  10.2  10.1  10.0           1.7  1.7  1.7  1.8
40  2005         10.1   9.8   9.9   9.6           1.8  1.8  1.9  2.0
41  2006          9.5   9.6   9.9   9.8           2.1  2.2  2.5  2.7
42  2007         10.1   9.5   9.8   9.6           2.8  2.6  2.7  2.8
43  2008         10.1  10.0   9.9  10.1           2.9  2.8  2.8  2.9
44  2009         10.1  10.6  11.1  10.7           2.7  2.5  2.6  2.7
45  2010         10.6  10.6  10.3   9.4           2.6  2.5  2.5  2.7
46  2011          9.7   9.2   9.8   9.4           2.6  2.5  2.4  2.3
47  2012          8.8   8.6   8.6   8.7           2.2  2.1  1.9  1.9
48  2013          8.6   8.2   8.3   8.2           2.1  1.9  1.9  2.1
49  2014          8.3   7.5   7.4   7.0           2.0  1.9  1.8  1.9
50  2015          7.1   6.8   7.3   7.0           1.9  1.8  1.9  1.9
51  2016          7.0   6.7   6.8   6.9           1.7  1.7  1.8  1.8
52  2017          7.0   7.3   7.5   6.9           1.7  1.5  1.6  1.6
53  2018          7.0   6.8   7.1   6.6           1.5  1.5  1.6  1.5
54  2019          7.0   6.8   6.8   6.4           1.4  1.3  1.4  1.4
55  2020          6.6   5.7   6.4   6.5           1.1  0.9  0.9  1.0
56

Insights
* The provided code sorts the DataFrame 'vacancy_df' in ascending order based on the 'year' column, effectively arranging the data chronologically by year. It also resets the index of the DataFrame to maintain a continuous integer index sequence.

In [30]:
# Reshaping the DataFrame by setting the 'year' as the index, stacking the columns, and resetting the index
vacancy_df = vacancy_df.set_index('year').stack().reset_index().rename(columns={'level_1': 'quarter'})

# Creating a 'date' column by combining 'year' and 'quarter' columns
vacancy_df['date'] = vacancy_df['year'] + '-' + vacancy_df['quarter']

# Dropping 'year' and 'quarter' columns
vacancy_df.drop(['year', 'quarter'], axis=1, inplace=True)

# Converting the 'date' column to datetime format
vacancy_df['date'] = pd.to_datetime(vacancy_df['date'])

# Setting the 'date' column as the new index
vacancy_df.set_index(['date'], inplace=True)

In [31]:
vacancy_df

,rent_vacancy,owner_vacancy
date,,
1965-01-01,8.5,1.7
1965-04-01,8.2,1.5
1965-07-01,7.8,1.6
1965-10-01,8.5,1.5
1966-01-01,8.3,1.5
...,...,...
2020-10-01,6.5,1.0
2021-01-01,6.8,0.9
2021-04-01,6.2,0.9


In [32]:
### upsample to monthly date using nearest neighbor

vacancy_monthly = vacancy_df.resample('M').nearest()
vacancy_monthly.head()

,rent_vacancy,owner_vacancy
date,,
1965-01-31,8.5,1.7
1965-02-28,8.2,1.5
1965-03-31,8.2,1.5
1965-04-30,8.2,1.5
1965-05-31,7.8,1.6


Insights
1. The code reshapes the DataFrame 'vacancy_df' by setting the 'year' column as the index, stacking the columns to create a multi-index, and then resetting the index to make the data more accessible.
2. It creates a new 'date' column by combining the 'year' and 'quarter' columns, facilitating time-based analysis.
3. The code converts the 'date' column to a datetime format and sets it as the new index, preparing the DataFrame for time series analysis with dates as the primary axis.

### [Employment Population Ratio](https://fred.stlouisfed.org/series/EMRATIO)

In [33]:
Em_ratio = pd.read_csv('https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=EMRATIO&scale=left&cosd=1948-01-01&coed=2022-07-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-22&revision_date=2022-08-22&nd=1948-01-01')

In [34]:
# Renaming the columns of the DataFrame to 'date' and 'emratio' for clarity
Em_ratio.columns = ['date', 'emratio']

# Converting the 'date' column to datetime format with the specified format
Em_ratio['date'] = pd.to_datetime(Em_ratio['date'], format='%Y-%m-%d')

# Setting the 'date' column as the index for time series analysis
Em_ratio.set_index(['date'], inplace=True)

In [35]:
Em_ratio

,emratio
date,
1948-01-01,56.6
1948-02-01,56.7
1948-03-01,56.1
1948-04-01,56.7
1948-05-01,56.2
...,...
2022-03-01,60.1
2022-04-01,59.9
2022-05-01,60.0


Insights
* the above code first rename the columns in the DataFrame to 'date' and 'emratio' for better readability and then convert the 'date' column to datetime format with the specified format ('%Y-%m-%d'). Finally, it sets the 'date' column as the DataFrame's index, likely for time series analysis with date-based data

### [Population Level](https://fred.stlouisfed.org/series/CNP16OV)

In [36]:
population_level = pd.read_csv('https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=CNP16OV&scale=left&cosd=1948-01-01&coed=2022-07-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-22&revision_date=2022-08-22&nd=1948-01-01')

In [37]:
# Renames the columns in the DataFrame to 'date' and 'pop_level'.
population_level.columns = ['date', 'pop_level']

# Converts the 'date' column to datetime format with the specified format and sets it as the index.
population_level['date'] = pd.to_datetime(population_level['date'], format='%Y-%m-%d')

population_level.set_index(['date'], inplace=True)

In [38]:
population_level

,pop_level
date,
1948-01-01,102603
1948-02-01,102698
1948-03-01,102771
1948-04-01,102831
1948-05-01,102923
...,...
2022-03-01,263444
2022-04-01,263559
2022-05-01,263679


Insights
* This code fetches population level data from a remote CSV file, renames the columns, converts the 'date' column to datetime format, and sets 'date' as the index for further analysis, likely related to population trends over time.

### [Gross Domestic Product](https://fred.stlouisfed.org/graph/?g=znfe)

In [39]:
gdp = pd.read_csv('https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=off&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=USALORSGPNOSTSAM&scale=left&cosd=1990-01-01&coed=2022-05-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-22&revision_date=2022-08-22&nd=1960-01-01')

In [40]:
gdp.columns = ['date', 'gdp']

gdp['date'] = pd.to_datetime(gdp['date'], format='%Y-%m-%d')

gdp.set_index(['date'], inplace=True)

In [41]:
gdp

,gdp
date,
1990-01-01,101.344728
1990-02-01,101.366177
1990-03-01,101.360289
1990-04-01,101.315362
1990-05-01,101.223169
...,...
2022-01-01,100.284923
2022-02-01,100.151473
2022-03-01,100.008122


### [Mortgage Rate in United States](https://fred.stlouisfed.org/graph/?g=zneW)

In [42]:
mortgage_rate = pd.read_csv('https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=MORTGAGE30US&scale=left&cosd=1990-01-01&coed=2022-08-18&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-22&revision_date=2022-08-22&nd=1971-04-02')

In [43]:
mortgage_rate.columns = ['date', 'mortgage_rate']

mortgage_rate['date'] = pd.to_datetime(mortgage_rate['date'], format='%Y-%m-%d')

mortgage_rate.set_index(['date'], inplace=True)

In [44]:
mortgage_rate

,mortgage_rate
date,
1990-01-01,9.8950
1990-02-01,10.1975
1990-03-01,10.2680
1990-04-01,10.3700
1990-05-01,10.4775
...,...
2022-04-01,4.9825
2022-05-01,5.2300
2022-06-01,5.5220


### [Federal Funds Effective Rate](https://fred.stlouisfed.org/series/DFF#0)

In [45]:
fed_fund_rate = pd.read_csv('https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=DFF&scale=left&cosd=1954-07-01&coed=2022-08-19&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Daily%2C%207-Day&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-22&revision_date=2022-08-22&nd=1954-07-01')

In [46]:
# Renames the columns in the DataFrame to 'date' and 'fed_fund_rate'.
fed_fund_rate.columns = ['date', 'fed_fund_rate']

# Converts the 'date' column to datetime format with the specified format and sets it as the index.
fed_fund_rate['date'] = pd.to_datetime(fed_fund_rate['date'], format='%Y-%m-%d')

fed_fund_rate.set_index(['date'], inplace=True)

In [47]:
fed_fund_rate

,fed_fund_rate
date,
1954-07-01,1.13
1954-07-02,1.25
1954-07-03,1.25
1954-07-04,1.25
1954-07-05,0.88
...,...
2022-08-15,2.33
2022-08-16,2.33
2022-08-17,2.33


Insights
* This code fetches Federal Funds Rate data from a remote CSV file, renames the columns, converts the 'date' column to datetime format, and sets 'date' as the index for further analysis, likely related to tracking changes in the Federal Funds Rate over time.


### [Delinquency Rate](https://fred.stlouisfed.org/series/DRSFRMACBS) Quaterly

In [48]:
delinquent_rate = pd.read_csv('https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=DRSFRMACBS&scale=left&cosd=1991-01-01&coed=2022-04-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Quarterly%2C%20End%20of%20Period&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-22&revision_date=2022-08-22&nd=1991-01-01')

In [49]:
delinquent_rate.columns = ['date', 'delinquent_rate']

delinquent_rate['date'] = pd.to_datetime(delinquent_rate['date'], format='%Y-%m-%d')

delinquent_rate.set_index(['date'], inplace=True)

In [50]:
delinquent_rate

,delinquent_rate
date,
1991-01-01,3.09
1991-04-01,3.18
1991-07-01,3.22
1991-10-01,3.28
1992-01-01,3.12
...,...
2021-04-01,2.47
2021-07-01,2.30
2021-10-01,2.28


###  [Real Disposable Personal Income](https://fred.stlouisfed.org/series/DSPIC96#0)

In [51]:
real_disp_income = pd.read_csv('https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=DSPIC96&scale=left&cosd=1959-01-01&coed=2022-06-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-22&revision_date=2022-08-22&nd=1959-01-01')

In [52]:
# Renames the columns in the DataFrame to 'date' and 'disp_income' for clarity.
real_disp_income.columns = ['date', 'disp_income']

# Converts the 'date' column to datetime format with the specified format and sets it as the index.
real_disp_income['date'] = pd.to_datetime(real_disp_income['date'], format='%Y-%m-%d')

real_disp_income.set_index(['date'], inplace=True)

In [53]:
real_disp_income

,disp_income
date,
1959-01-01,2191.5
1959-02-01,2198.1
1959-03-01,2210.7
1959-04-01,2225.0
1959-05-01,2237.1
...,...
2022-02-01,15125.6
2022-03-01,15064.1
2022-04-01,15055.2


Insights
* This code renames the columns in the DataFrame to 'date' and 'disp_income' for better readability and converts the 'date' column to datetime format, setting it as the index for further analysis, likely related to disposable income data over time.


### [Median usual weekly nominal earnings](https://fred.stlouisfed.org/series/LEU0252887700Q#0) Quaterly

In [54]:
week_earning = pd.read_csv('https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=LEU0252887700Q&scale=left&cosd=2000-01-01&coed=2022-04-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Quarterly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-22&revision_date=2022-08-22&nd=2000-01-01')

In [55]:
week_earning.columns = ['date', 'week_earning']

week_earning['date'] = pd.to_datetime(week_earning['date'], format='%Y-%m-%d')

week_earning.set_index(['date'], inplace=True)

In [56]:
week_earning

,week_earning
date,
2000-01-01,603
2000-04-01,606
2000-07-01,611
2000-10-01,614
2001-01-01,620
...,...
2021-04-01,1048
2021-07-01,1068
2021-10-01,1069


In [57]:
## linear interpolation to create monthly data

week_earning_monthly = week_earning.resample('MS').interpolate()

week_earning_monthly

,week_earning
date,
2000-01-01,603.000000
2000-02-01,604.000000
2000-03-01,605.000000
2000-04-01,606.000000
2000-05-01,607.666667
...,...
2021-12-01,1089.666667
2022-01-01,1100.000000
2022-02-01,1102.333333


### [Personal Saving](https://fred.stlouisfed.org/series/PMSAVE)

In [58]:
per_save = pd.read_csv('https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=PMSAVE&scale=left&cosd=1959-01-01&coed=2022-06-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-22&revision_date=2022-08-22&nd=1959-01-01')

In [59]:
per_save.columns = ['date', 'pm_save']

per_save['date'] = pd.to_datetime(per_save['date'], format='%Y-%m-%d')

per_save.set_index(['date'], inplace=True)

In [60]:
per_save

,pm_save
date,
1959-01-01,39.6
1959-02-01,37.5
1959-03-01,36.5
1959-04-01,40.0
1959-05-01,38.1
...,...
2022-02-01,813.3
2022-03-01,695.8
2022-04-01,651.1


### [Personal Consumption Expenditures Durable Goods](https://fred.stlouisfed.org/series/PCEDG)

In [61]:
consump_durable = pd.read_csv('https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=PCEDG&scale=left&cosd=1959-01-01&coed=2022-06-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-22&revision_date=2022-08-22&nd=1959-01-01')

In [62]:
consump_durable.columns = ['date', 'consump_durable']

consump_durable['date'] = pd.to_datetime(consump_durable['date'], format='%Y-%m-%d')

consump_durable.set_index(['date'], inplace=True)

In [63]:
consump_durable

,consump_durable
date,
1959-01-01,42.3
1959-02-01,44.2
1959-03-01,44.4
1959-04-01,45.1
1959-05-01,45.4
...,...
2022-02-01,2178.5
2022-03-01,2165.6
2022-04-01,2191.9


Insights
* This code renames the columns in the DataFrame to 'date' and 'consump_durable' for clarity. It then converts the 'date' column to datetime format using the specified format ('%Y-%m-%d') and sets 'date' as the index for further analysis, potentially related to durable goods consumption data over time

### [Home Ownership Rate](https://www.huduser.gov/portal/ushmc/hi_HOR.html) Quarterly

In [64]:
hor_qtr = pd.read_csv(r"C:\House_price\notebook\data\hpi_quarterly_data.csv")

In [65]:
hor_qtr

,date,hp_idx
0,1987-01-01,64.374667
1,1987-04-01,65.588333
2,1987-07-01,66.924667
3,1987-10-01,68.116333
4,1988-01-01,69.252667
...,...,...
137,2021-04-01,253.658000
138,2021-07-01,266.828000
139,2021-10-01,277.493667
140,2022-01-01,290.924667


### New Homes Sold

In [66]:
demand_hud = pd.read_excel('https://www.huduser.gov/portal/periodicals/ushmc/charts/HousingDemand.xlsx',
                          sheet_name='Sales_Monthly')

In [67]:
new_sold = demand_hud[['Month', 'NewSold_US']].copy()
new_sold.columns = ['date', 'new_sold']

new_sold['date'] = pd.to_datetime(new_sold['date'])

new_sold.set_index(['date'], inplace=True)

In [68]:
new_sold

,new_sold
date,
1963-01-01,591
1963-02-01,464
1963-03-01,461
1963-04-01,605
1963-05-01,586
...,...
2023-02-01,625
2023-03-01,640
2023-04-01,671


### [S&P/Case-Shiller U.S. National Home Price Index](https://fred.stlouisfed.org/series/CSUSHPISA)

In [69]:
hp_index = pd.read_csv('https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=CSUSHPISA&scale=left&cosd=1987-01-01&coed=2022-05-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-23&revision_date=2022-08-23&nd=1987-01-01')

In [70]:
hp_index.columns = ['date', 'hp_idx']

hp_index['date'] = pd.to_datetime(hp_index['date'])

hp_index.set_index(['date'], inplace=True)

In [71]:
hp_index

,hp_idx
date,
1987-01-01,63.965
1987-02-01,64.424
1987-03-01,64.735
1987-04-01,65.131
1987-05-01,65.563
...,...
2022-01-01,285.708
2022-02-01,290.795
2022-03-01,296.271


In [72]:
# downsampling to quarter period index

hp_index_qtr = hp_index.resample('QS').mean()

In [73]:
hp_index_qtr

,hp_idx
date,
1987-01-01,64.374667
1987-04-01,65.588333
1987-07-01,66.924667
1987-10-01,68.116333
1988-01-01,69.252667
...,...
2021-04-01,253.658000
2021-07-01,266.828000
2021-10-01,277.493667


### Combining All Above Data

#### Monthly dataset

In [74]:
### monthly data frame
monthly_df = [
            permits,
            building_permit_value,
            new_starts,
            completions,
            manufacturing,
            new_for_sale,
            Em_ratio,
            population_level,
            gdp,
            mortgage_rate,
            fed_fund_rate,
            real_disp_income,
            per_save,
            consump_durable,
            new_sold,
            ]

In [75]:
monthly_1990 = constr_spend.join(monthly_df, how='inner')

In [76]:
monthly_1990 = monthly_1990.loc[:'2020-12-01']


In [77]:
monthly_1990

,spend,permits,permit_val,starts,completions,manufactured,new_for_sale,months_supply,emratio,pop_level,gdp,mortgage_rate,fed_fund_rate,disp_income,pm_save,consump_durable,new_sold
date,,,,,,,,,,,,,,,,,
1993-01-01,194150,989,5383.028,1091,975.0,248,265,5.4,61.4,193962,99.750560,7.9925,2.66,7237.6,425.6,538.1,596
1993-02-01,194689,953,5844.533,1063,1043.0,248,263,5.3,61.4,194108,99.668158,7.6825,3.25,7271.8,441.5,524.2,604
1993-03-01,190185,881,8227.729,950,983.0,241,268,5.4,61.5,194248,99.585598,7.4950,3.65,7249.2,439.0,521.2,602
1993-04-01,198296,922,8918.858,1110,1067.0,241,270,4.7,61.5,194398,99.511168,7.4720,3.31,7286.8,436.4,543.0,701
1993-05-01,198835,911,8320.702,1128,973.0,240,273,5.3,61.7,194549,99.450667,7.4650,3.01,7276.3,415.8,552.0,626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-01,641103,1055,21995.756,1023,895.0,94,286,3.5,56.5,260558,97.297319,2.9350,0.10,15739.2,2714.5,1791.7,1029
2020-09-01,662353,1121,23179.135,1105,933.0,96,286,2.5,56.6,260742,97.433292,2.8900,0.09,15799.7,2569.3,1817.8,988
2020-10-01,684180,1141,23846.574,1162,898.0,100,284,3.5,57.4,260925,97.572374,2.8340,0.09,15729.1,2462.2,1826.1,1027


### Quarterly datasets

In [78]:
quarter_df = [vacancy_df,

            week_earning,

            delinquent_rate,

            hor_qtr]

In [79]:
qtr_2000 = vacancy_df.join(quarter_df[1:], how='inner')

In [80]:
qtr_2000

,rent_vacancy,owner_vacancy,week_earning,delinquent_rate,date,hp_idx


### Save Datasets in local folder

In [81]:
def save_the_data(d:pd.DataFrame, name:str, path:str):
    file = handle_file(path, name)
    d.to_csv(file, index_label='date')

In [82]:
save_the_data(monthly_1990, 'monthly_data_1990.csv', r'C:\House_price\notebook\data')

In [83]:
save_the_data(qtr_2000, 'quarterly_data_2000.csv', r'C:\House_price\notebook\data')

In [84]:
save_the_data(hp_index, 'hpi_monthly_data.csv', r'C:\House_price\notebook\data')